# Try scrapping stockx

Scraping pipeline:
1. Get entire catalog --> figure out individual product link
2. Product link --> sku, description
3. sku --> transaction data (activity)
4. Example Transaction data plot

In [ ]:
import requests
from bs4 import BeautifulSoup as bs4
import re
import json
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import phonenumbers
import pandas as pd
import csv
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
base_url = 'https://stockx.com'

In [ ]:
# predefined fuctions and settings

# To move columns in a pandas table
def movecol(df, cols_to_move=[], ref_col='', place='After'):

    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]

    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]

    return(df[seg1 + seg2 + seg3])
# return substring between start and end of input string s
def find_between(s, start, end):
  return (s.split(start))[1].split(end)[0]

# utility to save figure
def save_fig(fig_id, tight_layout=True):
    path = os.path.join(fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

# display all columns
pd.set_option('display.max_columns', None)

1. Getting URLs for individual snealer items from catalog

In [ ]:
all_data = []
def get_urls(url = base_url):
    end_url = '/sneakers?page='
    # empty dictionary to store urls
    urls = []
    # for each page in results page
    for page in range(1,25):
        # build url
        url = base_url + end_url + str(page)

        # retrieve urls
        rsp = requests.get(url,headers = { 'User-Agent': 'Opera/9.80 (Windows NT 5.1; U; en) Presto/2.10.289 Version/12.01', })
        body = bs4(rsp.text, 'html.parser')
        #print(body.prettify())
        listings = body.find_all('div', "tile Tile-c8u7wn-0 bCufAv")

        # store urls in list
        for listing in listings:
            urls.append(listing.a['href'])#('https://sfbay.craigslist.org' + listing.a['href'])

        time.sleep(1)  # seconds

    # write list to csv
    with open('urls.csv', 'w', newline='') as file:
        writer = csv.writer(file, delimiter=',')
        for row in urls:
            writer.writerow([row])

    return urls

# run script
urls = get_urls()
print(len(urls))


2. Getting SKU and product details from URLs

In [ ]:
f=open('urls.csv',"r")
urls = f.read().split('\n')
f.close()
urls[0]

In [ ]:

def get_SKU(urls):

    for i in range(924,len(urls)):
        # build url
        item_url = base_url + urls[i]

        sess = requests.session()

        headers = {
            "referer": "https://stockx.com/",
            "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'"
            }

        req = sess.get(item_url, headers = headers)
        fname = os.path.join('shoes',urls[i][1:] + '.txt')
        print(i,' ', req.status_code,' ',fname)
        if req.status_code != 200:
            print('scrapping failed')
            break
        soup = bs4(req.text)
        product = soup.findAll('div','product-view')

        f = open(fname,"w")
        f.write(str(product))
        f.close()
        time.sleep(3)  # seconds


# run script
get_SKU(urls)


In [ ]:
product_table = pd.DataFrame(columns = ['name', 'description','image_link', 'release_date', 'model', 'sku','color'])
product_table.head()

In [ ]:
shoe_dir = os.path.join(os.getcwd(),'shoes')

for filename in os.listdir(shoe_dir):
   try:
        with open(os.path.join(shoe_dir, filename), 'r') as f:
            content = f.read()
            x = content.split('{')[1]
            name = find_between(x,'\"name\":',',')[1:-1]
            desc = find_between(x,'\"description\":',',')[1:-1]
            image = find_between(x,'\"image\":',',')[1:-1]
            release_date= find_between(x,'\"releaseDate\":',',')[1:-1]
            model = find_between(x,'\"model\":',',')[1:-1]
            sku = find_between(x,'\"sku\":',',')[1:-1]
            color = find_between(x,'\"color\":',',')[1:-1]
            tmp = pd.DataFrame({'name': [name], 'description':[desc],'image_link': [image], 'release_date':[release_date], 'model':[model], 'sku':[sku],'color':[color]})
            product_table = product_table.append(tmp)
   except IndexError:
       print('Failed File: ', filename)
product_table.to_csv('product_table.csv')
print(product_table.shape)
product_table.head()

3. Scrapping Transaction data given SKU


In [ ]:
def text_to_df(str):
    df = pd.DataFrame()

    record = re.findall('\{.*?\}',str)
    #record = record[:2]
    #print(len(record))
    #print(record)
    for i in range(len(record)):
        dict = {}
        ss = record[i].split(',')
        #print(ss)
        for sss in ss:
            ssss= sss.split(':')
            dict[ssss[0][1:-1]] = ssss[1]
        tmp_df = pd.DataFrame(dict, index = [0])
        df = df.append(tmp_df)
    return df




In [ ]:
def get_activity(sku, p_name,idx):
    page = 1
    all_activity = ''
    while True:
        item_activity_url = base_url +'/api/products/'+sku + '/activity?state=480&currency=USD&limit=20000&page='+str(page)+'&sort=createdAt&order=DESC&country=US'
        item_activity_url
        sess = requests.session()
        #print(item_activity_url)
        headers = {
            "referer": base_url + '/' + p_name,
            "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'"
            }

        req = sess.get(item_activity_url, headers = headers)
        print(idx,' ',req.status_code,' ', p_name)
        content= req.text
        activity = find_between(content,'[',']')
        all_activity = all_activity + activity

        next_page = re.search('\"nextPage\"\:.*?,',content).group(0).split(':')[1]
        #print(next_page)

        if next_page == 'null,':
            break
        page +=1

    #print(len(all_activity))
    activity_df = text_to_df(all_activity)
    return activity_df

In [ ]:
product_table = pd.read_csv('product_table.csv')
for index,row in product_table.iterrows():
    if index >= 0: # index to start
        try:
            sku, p_name = row['sku'], row['name']
            product_activity = get_activity(sku,p_name, index)
            product_activity['product_name'] = p_name
            product_activity.to_csv(os.path.join('activity',p_name+'activity.csv'))
        except FileNotFoundError:
           print('Failed File: ', p_name)

4. Example plot of Transaction Activity

In [ ]:
activity_df=pd.read_csv('activity_df.csv')
activity_df['amount'] = activity_df['amount'].astype('float')
activity_df['createdAt'] = activity_df['createdAt'].str.strip('\"')
activity_df['shoeSize'] = activity_df['shoeSize'].str.strip('\"')
activity_df['shoeSize'] = activity_df['shoeSize'].astype('float')
activity_df.head()

In [ ]:
activity_df['createdAt']=pd.to_datetime(activity_df['createdAt'])
activity_df.info()

In [ ]:
plt.figure(figsize=(20,10))
sns.set_palette("husl")
chart = sns.lineplot(x = activity_df['createdAt'], y = activity_df['amount'], hue = 'shoeSize',palette='RdBu', data = activity_df)
plt.title('Jordan 1 Retro High Court Purple White')
#chart.set_xticklabels(rotation=65, horizontalalignment='right')
save_fig('activity_test_fig')

